<a href="https://colab.research.google.com/github/Vayansh/Multi_object_detection_in_a_image/blob/main/Multi_object_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kaggle
from google.colab import files

files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d dronevision/vsaiv1


Saving kaggle.json to kaggle.json
100% 12.8G/12.8G [11:24<00:00, 20.4MB/s]
100% 12.8G/12.8G [11:24<00:00, 20.1MB/s]


In [ ]:
!unzip '/content/vsaiv1.zip' -d '/content/drive/MyDrive/Multi-Object_dataset'

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
# Training dataset
dataset_file = open('dataset_file.csv','w+')
for i in os.listdir('/content/drive/MyDrive/Multi-Object_dataset/VSAIv1/split_ss_444_lsv/train/annfiles'):
    f = open('/content/drive/MyDrive/Multi-Object_dataset/VSAIv1/split_ss_444_lsv/train/annfiles/'+i,'r')
    dataset_file.write(i[:-3]+'png,')
    c = f.read(1)
    while c:
        if c == ' ':
            dataset_file.write(',')
        elif c == '\n':
            dataset_file.write(c+i[:-3]+'png,')
        else:
            dataset_file.write(c)
        c = f.read(1)
    f.close()
    dataset_file.write('\n')
dataset_file.close()

In [ ]:
# validation dataset
dataset_file = open('val_dataset_file.csv','w+')
for i in os.listdir('/content/drive/MyDrive/Multi-Object_dataset/VSAIv1/split_ss_444_lsv/val/annfiles'):
    f = open('/content/drive/MyDrive/Multi-Object_dataset/VSAIv1/split_ss_444_lsv/val/annfiles/'+i,'r')
    dataset_file.write(i[:-3]+'png,')
    c = f.read(1)
    while c:
        if c == ' ':
            dataset_file.write(',')
        elif c == '\n':
            dataset_file.write(c+i[:-3]+'png,')
        else:
            dataset_file.write(c)
        c = f.read(1)
    f.close()
    dataset_file.write('\n')
dataset_file.close()

In [ ]:
# Training data
df = pd.read_csv('dataset_file.csv',names = ['filename','bbx1','bby1','bbx2','bby2','bbx3','bby3','bbx4','bby4','label1','label2'])
df.dropna(inplace = True)
df.head(10)

,filename,bbx1,bby1,bbx2,bby2,bbx3,bby3,bbx4,bby4,label1,label2
0,r11_DJI_0009__1024__2472___0.png,671.10010,675.20000,595.19995,647.30000,669.50000,445.10000,745.30005,472.90000,large-vehicle,0.0
1,r11_DJI_0009__1024__2472___0.png,810.19995,627.70000,714.60010,593.60000,845.00000,228.40000,940.60010,262.50000,large-vehicle,0.0
2,r11_DJI_0009__1024__2472___0.png,929.60010,100.40000,963.89990,143.80000,858.50000,226.90000,824.30005,183.50000,small-vehicle,2.0
3,r11_DJI_0009__1024__2472___0.png,1045.60010,0.00000,1023.30005,42.40000,899.80005,0.50000,915.10010,0.00000,small-vehicle,2.0
5,q_DJI_0792__1024__2976___1648.png,245.39990,611.60010,247.69995,635.50000,200.19995,640.19995,197.80005,616.30005,small-vehicle,2.0
7,r4_DJI_0616__1024__2472___824.png,754.30005,1002.00000,751.89990,991.40000,772.50000,986.80005,774.89990,997.40000,small-vehicle,0.0
8,r4_DJI_0616__1024__2472___824.png,786.89990,996.19995,784.30005,986.10000,802.39990,981.50000,805.00000,991.50000,small-vehicle,0.0
10,r11_DJI_0009__1024__2976___824.png,751.60010,852.69995,611.69995,864.50000,581.89990,511.59998,721.80005,499.80005,large-vehicle,2.0
11,r11_DJI_0009__1024__2976___824.png,1024.00000,863.10000,957.19995,863.69995,944.19995,600.10000,1024.00000,596.60000,large-vehicle,1.0
13,q_DJI_0797__1024__824___1976.png,394.69995,994.50000,362.50000,1002.00000,346.80005,934.69995,379.00000,927.19995,small-vehicle,0.0


In [ ]:
# val data
df_val = pd.read_csv('val_dataset_file.csv',names = ['filename','bbx1','bby1','bbx2','bby2','bbx3','bby3','bbx4','bby4','label1','label2'])
df_val.dropna(inplace = True)
df_val.head(10)

,filename,bbx1,bby1,bbx2,bby2,bbx3,bby3,bbx4,bby4,label1,label2
1,f1_DJI_0777__1024__1648___1976.png,345.80005,15.800049,358.09998,5.400024,373.19995,23.300049,360.90002,33.69995,small-vehicle,0.0
2,f1_DJI_0777__1024__1648___1976.png,488.50000,247.600100,474.50000,252.699950,465.00000,226.500000,479.00000,221.50000,small-vehicle,0.0
3,f1_DJI_0777__1024__1648___1976.png,505.50000,284.899900,491.39990,290.300050,481.50000,264.399900,495.60010,259.00000,small-vehicle,0.0
4,f1_DJI_0777__1024__1648___1976.png,281.59998,397.000000,275.90002,378.399900,310.40002,367.899900,316.09998,386.50000,small-vehicle,0.0
5,f1_DJI_0777__1024__1648___1976.png,866.19995,211.199950,867.30005,229.600100,832.60010,231.699950,831.50000,213.39990,small-vehicle,2.0
8,g_DJI_0577__1024__1648___824.png,525.80005,926.100000,554.00000,950.900000,499.89990,1012.300050,471.69995,987.50000,small-vehicle,2.0
9,g_DJI_0577__1024__1648___824.png,488.30005,837.199950,511.19995,812.800050,563.50000,861.699950,540.50000,886.19995,small-vehicle,0.0
10,g_DJI_0577__1024__1648___824.png,505.30005,795.100000,530.19995,767.000000,597.19995,826.300050,572.39990,854.40000,small-vehicle,0.0
11,g_DJI_0577__1024__1648___824.png,549.60010,758.199950,572.60010,728.500000,637.19995,778.699950,614.19995,808.40000,small-vehicle,2.0
12,g_DJI_0577__1024__1648___824.png,879.69995,526.500000,911.50000,546.300050,868.30005,615.800050,836.39990,595.90000,small-vehicle,0.0


In [ ]:
img = cv.imread('/content/drive/MyDrive/Multi-Object_dataset/VSAIv1/split_ss_444_lsv/train/images/q_DJI_0877__1024__0___0.png')
print(img.shape)

(1024, 1024, 3)


In [ ]:
df['label1'].unique()

array(['large-vehicle', 'small-vehicle'], dtype=object)

In [ ]:
# training dataset
labels = []
bboxes = []
imagePaths = []
for i in df.itertuples(index=True, name='Pandas'):
    imagePaths.append(i.filename)
    bboxes.append([i.bbx2/1024,i.bby2/1024,i.bbx4/1024,i.bby4/1024])
    if i.label1 == 'small-vehicle':
        labels.append(0)
    else:
        labels.append(1)


In [ ]:
# val dataset
val_labels = []
val_bboxes = []
val_imagePaths = []
for i in df_val.itertuples(index=True, name='Pandas'):
    val_imagePaths.append(i.filename)
    val_bboxes.append([i.bbx2/1024,i.bby2/1024,i.bbx4/1024,i.bby4/1024])
    if i.label1 == 'small-vehicle':
        val_labels.append(0)
    else:
        val_labels.append(1)

In [ ]:
print(len(bboxes))
print(len(imagePaths))

47842
47842


In [ ]:
train_X = pd.DataFrame({'path':imagePaths,'bboxes':bboxes,'labels':labels})
train_X.head(5)

,path,bboxes,labels
0,r11_DJI_0009__1024__2472___0.png,"[0.581249951171875, 0.63212890625, 0.727832080...",1
1,r11_DJI_0009__1024__2472___0.png,"[0.69785166015625, 0.5796875, 0.91855478515625...",1
2,r11_DJI_0009__1024__2472___0.png,"[0.94130849609375, 0.1404296875, 0.80498051757...",0
3,r11_DJI_0009__1024__2472___0.png,"[0.999316455078125, 0.04140625, 0.893652441406...",0
4,q_DJI_0792__1024__2976___1648.png,"[0.241894482421875, 0.62060546875, 0.193164111...",0


In [ ]:
val_X = pd.DataFrame({'path': val_imagePaths,'bboxes' : val_bboxes,'labels':val_labels})

In [ ]:
val_X.head(10)

,path,bboxes,labels
0,f1_DJI_0777__1024__1648___1976.png,"[0.34970701171875, 0.005273461328125, 0.352441...",0
1,f1_DJI_0777__1024__1648___1976.png,"[0.46337890625, 0.246777294921875, 0.467773437...",0
2,f1_DJI_0777__1024__1648___1976.png,"[0.47988271484375, 0.283496142578125, 0.483984...",0
3,f1_DJI_0777__1024__1648___1976.png,"[0.26943361328125, 0.36953115234375, 0.3086913...",0
4,f1_DJI_0777__1024__1648___1976.png,"[0.846972705078125, 0.22421884765625, 0.812011...",0
5,g_DJI_0577__1024__1648___824.png,"[0.541015625, 0.92861328125, 0.460644482421875...",0
6,g_DJI_0577__1024__1648___824.png,"[0.499218701171875, 0.793750048828125, 0.52783...",0
7,g_DJI_0577__1024__1648___824.png,"[0.517773388671875, 0.7490234375, 0.5589842773...",0
8,g_DJI_0577__1024__1648___824.png,"[0.55917978515625, 0.71142578125, 0.5998046386...",0
9,g_DJI_0577__1024__1648___824.png,"[0.89013671875, 0.533496142578125, 0.816796777...",0


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator(validation_split = 0)
train_generator = generator.flow_from_dataframe(train_X,directory='/content/drive/MyDrive/Multi-Object_dataset/VSAIv1/split_ss_444_lsv/train/images',  target_size = (1024,1024), class_mode = 'multi_output',
                                                x_col = 'path' , y_col= ['bboxes','labels'],batch_size=10)

Found 47842 validated image filenames.


In [ ]:
generator2 = ImageDataGenerator(validation_split = 0)
val_generator = generator2.flow_from_dataframe(val_X,directory='/content/drive/MyDrive/Multi-Object_dataset/VSAIv1/split_ss_444_lsv/val/images',  target_size = (1024,1024), class_mode = 'multi_output',
                                                x_col = 'path' , y_col= ['bboxes','labels'],batch_size=10)

Found 13285 validated image filenames.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(1024,1024,3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)


# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid",name="bounding_box")(bboxHead)
# construct a second fully-connected layer head, this one to predict
# the class label

softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(1, activation="softmax",name="class_label")(softmaxHead)
# put together our model which accept an input image and then output
# bounding box coordinates and a class label
model = Model(inputs=vgg.input,outputs=(bboxHead, softmaxHead))


58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
losses = {
"class_label": "binary_crossentropy",
"bounding_box": "mean_squared_error",
}
# define a dictionary that specifies the weights per loss (both the
# class label and bounding box outputs will receive equal weight)
lossWeights = {
"class_label": 1.0,
"bounding_box": 1.0
}


opt = Adam(lr=0.0001)
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024, 1024,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, 1024, 1024,   1792        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 block1_conv2 (Conv2D)          (None, 1024, 1024,   36928       ['block1_conv1[0][0]']           
                                64)                                                           

In [ ]:
from keras.callbacks import ModelCheckpoint


In [ ]:
model.load_weights("/content/drive/MyDrive/Multi-Object_dataset/multi-object_checkpoint.h5")

In [ ]:
filepath= "/content/drive/MyDrive/Multi-Object_dataset/multi-object_checkpoint.h5"
checkpoint = ModelCheckpoint(filepath)
callbacks_list = [checkpoint]

In [ ]:
model.fit(train_generator, validation_data= val_generator,epochs = 10, callbacks=callbacks_list)

Epoch 1/10
4785/4785 [==============================] - ETA: 0s - loss: 12.8327 - bounding_box_loss: 0.3356 - class_label_loss: 12.4970 - bounding_box_accuracy: 0.2201 - class_label_accuracy: 0.0385

In [ ]:
model.save_weights('')

In [ ]:
model.predict('')